## Momentum Investing Strategy
Select 50 stocks with highest price momentum

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
# Check one year price return of each stocks
symbol = 'AAPL'

# Making API call
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [5]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2097128304209,
 'week52high': 147.75,
 'week52low': 61.9,
 'week52highSplitAdjustOnly': 145.15,
 'week52lowSplitAdjustOnly': 62.79,
 'week52change': 1.084544766171328,
 'sharesOutstanding': 17464332404,
 'float': 0,
 'avg10Volume': 91347727,
 'avg30Volume': 110271034,
 'day200MovingAvg': 125.03,
 'day50MovingAvg': 127.2,
 'employees': 148971,
 'ttmEPS': 3.8,
 'ttmDividendRate': 0.8305168988341337,
 'dividendYield': 0.00671299555723256,
 'nextDividendDate': '0',
 'exDividendDate': '2021-02-05',
 'nextEarningsDate': '2021-04-15',
 'peRatio': 33.708200375215014,
 'beta': 1.22634337135288,
 'maxChangePercent': 47.17345080511124,
 'year5ChangePercent': 3.9340186099729686,
 'year2ChangePercent': 1.5858760203029942,
 'year1ChangePercent': 1.073362438850549,
 'ytdChangePercent': -0.0743715105065795,
 'month6ChangePercent': 0.09615122350321062,
 'month3ChangePercent': -0.05028164558236482,
 'month1ChangePercent': 0.024511677026558668,
 'day30ChangePerc

### Executing batch API call to IEX for fetching all stock data 

In [6]:
def get_chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [7]:
symbol_groups = list(get_chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])
    
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [8]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy


In [9]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [10]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,133.72,0.835062,N/A
1,AAL,24.96,1.58239,N/A
2,AAP,193.85,1.21173,N/A
3,AAPL,126.00,1.06123,N/A
4,ABBV,109.72,0.570581,N/A
...,...,...,...,...
500,YUM,114.00,0.804004,N/A
501,ZBH,163.00,0.768609,N/A
502,ZBRA,513.62,1.7725,N/A
503,ZION,56.63,1.39056,N/A


In [11]:
# Removing Low Momentum Stocks
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,63.30,4.60961,N/A
1,FCX,34.50,4.50302,N/A
2,GPS,31.14,4.32761,N/A
3,KSS,59.78,4.18923,N/A
4,MGM,41.30,2.878,N/A
5,VIAC,46.01,2.79508,N/A
6,TPR,41.52,2.74323,N/A
7,URI,342.20,2.70618,N/A
8,EXPE,180.54,2.68329,N/A
9,RCL,87.46,2.60858,N/A


In [13]:
# Calculating Number of shares to Buy
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio : ")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again!")
        portfolio_size = input("Enter the value of your portfolio : ")

In [14]:
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio : 100000
100000


In [15]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,63.30,4.60961,30
1,FCX,34.50,4.50302,56
2,GPS,31.14,4.32761,62
3,KSS,59.78,4.18923,32
4,MGM,41.30,2.878,47
5,VIAC,46.01,2.79508,42
6,TPR,41.52,2.74323,47
7,URI,342.20,2.70618,5
8,EXPE,180.54,2.68329,10
9,RCL,87.46,2.60858,22


### Build more realistic Momentum Strategy 

In [16]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

In [17]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [22]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.50,N/A,0.825904,N/A,0.294562,N/A,0.079422,N/A,0.115318,N/A,N/A
1,AAL,24.35,N/A,1.57578,N/A,0.871522,N/A,0.602734,N/A,0.128538,N/A,N/A
2,AAP,188.06,N/A,1.21892,N/A,0.204465,N/A,0.189213,N/A,0.128078,N/A,N/A
3,AAPL,128.00,N/A,1.08314,N/A,0.093978,N/A,-0.0502324,N/A,0.0242267,N/A,N/A
4,ABBV,109.97,N/A,0.560498,N/A,0.305948,N/A,0.0434162,N/A,0.0282737,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.10,N/A,0.782633,N/A,0.165629,N/A,0.0433724,N/A,0.0622553,N/A,N/A
501,ZBH,165.00,N/A,0.769288,N/A,0.171908,N/A,0.0476977,N/A,0.0262877,N/A,N/A
502,ZBRA,505.22,N/A,1.77526,N/A,0.903023,N/A,0.315265,N/A,0.0264183,N/A,N/A
503,ZION,56.62,N/A,1.37759,N/A,0.900927,N/A,0.303255,N/A,0.0279499,N/A,N/A
